In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def scrape_whatsapp(phone_number):
    url = f"https://web.whatsapp.com/{phone_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    registered = "No"
    if soup.find('div', {'class': '_1WZqU PNlAR'}):
        registered = "Yes"

    name_element = soup.find('span', {'class': '_1wjpf'})
    name = name_element.text if name_element else None

    status_element = soup.find('div', {'class': '_2hqOq _3xI7T'})
    status = status_element.text if status_element else None

    last_seen_element = soup.find('div', {'class': '_3H4MS'})
    last_seen = last_seen_element.text if last_seen_element else None

    profile_pic_element = soup.find('div', {'class': '_2ruVH'})
    profile_pic = profile_pic_element.img['src'] if profile_pic_element else None

    return {
        "Registered": registered,
        "Name": name,
        "Status": status,
        "Last Seen": last_seen,
        "Profile Picture": profile_pic
    }

def scrape_eyecon(phone_number):
    url = f"https://www.eyecon-app.com/{phone_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    registered = "No"
    if soup.find('div', {'id': 'fbProfileCover'}):
        registered = "Yes"

    name_element = soup.find('span', {'class': 'fullname'})
    name = name_element.text if name_element else None

    username_element = soup.find('span', {'class': 'username'})
    username = username_element.text if username_element else None

    profile_url_element = soup.find('meta', {'property': 'og:url'})
    profile_url = profile_url_element['content'] if profile_url_element else None

    return {
        "Registered": registered,
        "Name": name,
        "Username": username,
        "Profile URL": profile_url
    }
def scrape_truecaller(phone_number):
    url = f"https://www.truecaller.com/search/in/{phone_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    registered_element = soup.find('a', {'class': 'profile-sidebar-title'})
    registered = registered_element.text if registered_element else None

    name_element = soup.find('h1', {'class': 'profile-title'})
    name = name_element.text if name_element else None

    email_element = soup.find('a', {'class': 'email'})
    email = email_element.text if email_element else None

    return {
        "Registered": registered,
        "Name": name,
        "Email id": email
    }



def scrape_gpay(phone_number):
    url = f"https://gpay.app.goo.gl/{phone_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    registered_element = soup.find('div', {'class': 'DfTQ5d'})
    registered = registered_element.text if registered_element else None

    name = registered
    upi_id_elements = soup.find_all('div', {'class': 'DfTQ5d'})
    upi_ids = [element.text for element in upi_id_elements]

    return {
        "Registered": registered,
        "Name": name,
        "UPI IDs": upi_ids
    }


phone_numbers = [
    "+91 8268291167",
    "+91 9867913757",
    "+91 8779278482"
]

results = []

for phone_number in phone_numbers:
    whatsapp_data = scrape_whatsapp(phone_number)
    results.append(("WhatsApp", whatsapp_data))

    eyecon_data = scrape_eyecon(phone_number)
    results.append(("Eyecon", eyecon_data))

    truecaller_data = scrape_truecaller(phone_number)
    results.append(("Truecaller", truecaller_data))

    gpay_data = scrape_gpay(phone_number)
    results.append(("Gpay", gpay_data))

df = pd.DataFrame(results, columns=["Platform", "Data"])
print(df)   


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def scrape_data(phone_number, email):
    platforms = {
        "WhatsApp": {
            "url": f"https://web.whatsapp.com/{phone_number}",
            "selectors": {
                "registered": ('div', {'class': '_1WZqU PNlAR'}),
                "name": ('span', {'class': '_1wjpf'}),
                "status": ('div', {'class': '_2hqOq _3xI7T'}),
                "last_seen": ('div', {'class': '_3H4MS'}),
                "profile_picture": ('div', {'class': '_2ruVH'}),
                "upi_id": ('div', {'class': '_1MZWu'}),
                "username": ('div', {'class': '_1fQZE'}),
                "profile_url": ('div', {'class': '_1zGQT'})
            }
        },
        "Eyecon": {
            "url": f"https://www.eyecon-app.com/{phone_number}",
            "selectors": {
                "registered": ('div', {'id': 'fbProfileCover'}),  
                "name": ('span', {'class': 'fullname'}),
                "username": ('span', {'class': 'username'}),
                "profile_url": ('meta', {'property': 'og:url'})
            }
        },
        "Truecaller": {
            "url": f"https://www.truecaller.com/search/in/{phone_number}",
            "selectors": {
                "registered": ('a', {'class': 'profile-sidebar-title'}),
                "name": ('h1', {'class': 'profile-title'}),
                "email": ('a', {'class': 'email'})
            }
        },

        "Gpay": {
            "url": f"https://gpay.app.goo.gl/{phone_number}",
            "selectors": {
                "registered": ('div', {'class': 'DfTQ5d'}),
                "name": ('div', {'class': 'DfTQ5d'}),
                "upi_id": ('div', {'class': 'DfTQ5d'})
            }
        }
    }

    output = []
    for platform, data in platforms.items():
        url = data['url']
        selectors = data['selectors']

        # Make a request to the platform's URL
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the desired attributes using the provided selectors
        attributes = {}
        for attribute, selector in selectors.items():
            element = soup.find(*selector)
            attributes[attribute] = element.text if element else None

        # Add the attributes to the output list
        output.append({"Platform": platform, **attributes})

    # Create a DataFrame from the output list
    df = pd.DataFrame(output)

    return df


# Example usage
phone_number = "8928408593"
email_address = "pc8171468@gmail.com"
df = scrape_data(phone_number, email_address)
print(df)